# Evaluating the impact of curriculum learning on the training process for an intelligent agent in a videogame
Maestría en Ingeniería de Sistemas y Computación, Modalidad Profundización. 
Universidad Nacional de Colombia - Sede Bogotá

### Abstract:
Curriculum learning is a technique that can increase the learning speed of an intelligent agent that is learning a behavior policy to play a video game in an optimal way using reinforcement learning. It can be seen as a special form of transfer learning and is based on the idea of training using easier examples first, then using the harder complex ones later, to guide the optimization process. This technique is tested in a 3D 1-vs-1 soccer video game running in Unity, where two identical agents are trained using Proximal Policy Optimization, the training process for the second agent uses a 5-step curriculum designed to speed-up the learning rate.

### Keywords
Reinforcement Learning, Curriculum Learning, Continuation Method, Sparse Rewards, Deep Learning, Proximal Policy Optimization, Video Games, Unity Machine Learning Agents, Unity ML-Agents SDK, Unity Engine, Game AI

### Links:
* http://bit.ly/rsaenzi-master-thesis
* https://github.com/rsaenzi/master-thesis
* https://sites.google.com/view/rsaenzi-master-thesis/home
* https://www.mendeley.com/community/rigobertosaenz-referencias/


# Google Colab only

In [ ]:
# Delete colab sample data
!rm -R sample_data

# Google Compute Engine only

In [ ]:
# Make contents directory
#!mkdir -p 'content'
#!cd 'content'
#!pwd

# Paths
Directory paths that will be used for installing Unity and running experiments

In [ ]:
#@title Unity Project
UNITY_PROJECT_PATH = '/content/master-thesis/SoccerAcademy' #@param {type:"string"}
BUILD_PATH = '/content/soccer-academy-build/' #@param {type:"string"}
BUILD_NAME = 'SoccerAcademy.x86_64' #@param {type:"string"}

import os
FULL_BINARY_PATH = os.path.join(BUILD_PATH, BUILD_NAME)
print("Full Linux Binary Path: {}".format(FULL_BINARY_PATH))

In [ ]:
#@title Experiments
PARALLEL_EXECUTIONS = 1 #@param {type:"integer"}
EXPERIMENTS_NAME = 'PPO_curriculaA' #@param {type:"string"}
EXPERIMENTS_PATH = '/content/' #@param {type:"string"}

TRAINING_CONFIG_FILE = 'TrainingConfigPPO.yaml' #@param {type:"string"}
CURRICULA_FILE = 'TrainingCurriculaA.yaml' #@param {type:"string"}

TRAINING_CONFIG_PATH = os.path.join(EXPERIMENTS_PATH, TRAINING_CONFIG_FILE)
CURRICULA_PATH = os.path.join(EXPERIMENTS_PATH, CURRICULA_FILE)

print("Full TrainingConfig Path: {}".format(TRAINING_CONFIG_PATH))
print("Full Curricula Path: {}".format(CURRICULA_PATH))

# Linux Instance

Get all possible information we can from the Linux instance that run this notebook

In [ ]:
# Linux version
!cat /proc/version

In [ ]:
# Current working directory
!pwd

In [ ]:
# Directory contents
!ls

In [ ]:
# Python Version
!python3 --version

In [ ]:
# PIP version
!pip3 --version

In [ ]:
# CPU info
!cat /proc/cpuinfo

In [ ]:
# RAM info
!cat /proc/meminfo

# Install all Linux dependencies for Unity

This is a list of all possible things that Unity ML-Agents might need

In [ ]:
# PIP Python package manager
#!python3 -m pip install --upgrade pip
!pip install --upgrade pip

In [ ]:
# Tensorflow
#!pip install tensorflow
!pip install --upgrade tensorflow

In [ ]:
# Tensorboard
#!pip install tensorboard
!pip install --upgrade tensorboard

In [ ]:
# TensorboardColab
#!pip install tensorboardcolab
!pip install --upgrade tensorboardcolab

In [ ]:
# Keras
#!pip install keras
!pip install --upgrade keras

In [ ]:
# SetupTools
#!pip install setuptools
!pip install --upgrade setuptools

In [ ]:
# mlagents
#!pip install mlagents
!pip install --upgrade mlagents==0.16.1

In [ ]:
# Unity dependencies
!apt -qq update
!apt -qq install npm
!apt -qq install xvfb
!apt -qq install libgconf-2-4
!apt -qq install gconf-service lib32gcc1 lib32stdc++6 libasound2 libc6 libc6-i386 libcairo2 libcap2 libcups2 libdbus-1-3 libexpat1 libfontconfig1 libfreetype6 libgcc1 libgconf-2-4 libgdk-pixbuf2.0-0 libgl1-mesa-glx libglib2.0-0 libglu1-mesa libgtk2.0-0 libnspr4 libnss3 libpango1.0-0 libstdc++6 libx11-6 libxcomposite1 libxcursor1 libxdamage1 libxext6 libxfixes3 libxi6 libxrandr2 libxrender1 libxtst6 zlib1g debconf
!apt -qq install nodejs-dev node-gyp libssl1.0-dev

!apt-get update
!apt-get install pulseaudio

# Imports
Required library to run subsequent commands of this notebook

In [ ]:
# Required imports
import os
import tensorboard
import tensorflow as tf

from google.colab import drive
from psutil import virtual_memory
from tensorflow.python.client import device_lib

# Detailed instance info
Check hardware configuration using installed libraries

In [ ]:
# CPU info
device_lib.list_local_devices()

In [ ]:
# GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# RAM info
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

# Clone ``ml-agents``
To get Unity ML-Agents library

In [ ]:
# Get ml-agents from github repo
!git clone -b release_2_branch https://github.com/Unity-Technologies/ml-agents.git
print("Unity ml-agents repo cloned!")

# Clone ``master-thesis``
To get SoccerAcademy project in which experiment will be running

In [ ]:
# Get master-thesis from github repo
!git clone -b master https://github.com/rsaenzi/master-thesis.git
print("SoccerAcademy unity project cloned!")

# Get Unity Installer for Linux
Downloading the latest Unity Installer for Linux and make it executable:
https://forum.unity.com/threads/unity-on-linux-release-notes-and-known-issues.350256/

In [ ]:
# Download latest installer version for Linux
!wget http://beta.unity3d.com/download/292b93d75a2c/UnitySetup-2019.1.0f2 -O UnityInstaller
print("Unity installed for Linux downloaded!")

In [ ]:
# Make the downloaded file executable
!chmod +x UnityInstaller

# Unity Installation Config
We can configure which Unity exportation platforms will be installed.
Note: Don't change the ``INSTALL_LOCATION``. 

In [ ]:
#@title Choose what components to Install
# Components = "Unity" #@param ["Unity", "Windows-Mono", "Mac-Mono", "iOS", "Android", "WebGL", "FacebookGames"] {allow-input: true}

INSTALL_LOCATION = "/opt" #@param {type:"string"}

Install_Facebook = 0 #@param {type:"slider", min:0, max:1, step:1}
Install_MACOS = 0 #@param {type:"slider", min:0, max:1, step:1}
Install_Windows = 0 #@param {type:"slider", min:0, max:1, step:1}
Install_Android = 0 #@param {type:"slider", min:0, max:1, step:1}
Install_WebGL = 0 #@param {type:"slider", min:0, max:1, step:1}
Install_iOS = 0 #@param {type:"slider", min:0, max:1, step:1}

SELECTED_COMPS = "Unity"

SELECTED_COMPS += ",Facebook-Games" if Install_Facebook == 1 else ""
SELECTED_COMPS += ",Windows-Mono" if Install_Windows == 1 else ""
SELECTED_COMPS += ",Mac-Mono" if Install_MACOS == 1 else ""
SELECTED_COMPS += ",Android" if Install_Android == 1 else ""
SELECTED_COMPS += ",WebGL" if Install_WebGL == 1 else ""
SELECTED_COMPS += ",iOS" if Install_iOS == 1 else ""


UNITY_EXEC = INSTALL_LOCATION + '/Editor/Unity'

print("Selected Components are: {}".format(SELECTED_COMPS))
print("Unity Installer Binary Location: {}".format(UNITY_EXEC))

# Install Unity
You will be prompted to accept the Unity Terms of Service before starting the installation process, please enter 'y' on the textbox to accept the terms of the License Agreement.

In [ ]:
!./UnityInstaller --help

In [ ]:
!./UnityInstaller --unattended --components=$SELECTED_COMPS --install-location=$INSTALL_LOCATION
print("Unity installation completed!")

# Activate Unity
Before using Unity, we have to activate a license. Run the following code to create a license request file (``.alf``)

In [ ]:
#!xvfb-run --auto-servernum $UNITY_EXEC -nographics -logFile -batchmode -createManualActivationFile
#print("Unity license request file created!")

Open [Unity's Manual activation](https://license.unity3d.com/) website to upload the license request file (``.ulf``). Follow the steps to download the License file (``.x.ulf``), then upload it to **colab** using the left side-bar.

**Note** The License file (``.x.ulf``) can be reused to activate Unity in other colab sessions.

In [ ]:
!cp -r "/content/master-thesis/UnityLicense/Unity_v2019.x.ulf" '/content/'
!mv 'Unity_v2019.x.ulf' 'UnityLicense.x.ulf'
print("Unity license file copied to working directory!")

In [ ]:
!xvfb-run --auto-servernum $UNITY_EXEC -nographics -logFile -batchmode -manualLicenseFile UnityLicense.x.ulf
print("Unity license file installed!")

In [ ]:
!cd /content/ml-agents/ml-agents-envs && pip install -e .
!cd /content/ml-agents/ml-agents && pip install -e .
print("Unity ml-agents dependencies installed!")

# Update Unity API
Now we update the Unity API to prevent build errors. This operation requires to open the SoccerAcademy unity project.

In [ ]:
!xvfb-run --auto-servernum $UNITY_EXEC -nographics -logFile -batchmode -accept-apiupdate -buildTarget Linux64 -projectPath $UNITY_PROJECT_PATH -quit
print("Unity API update finished!")

# Build SoccerAcademy project
Creates the Linux executable to run the experiments

In [ ]:
!xvfb-run --auto-servernum $UNITY_EXEC -nographics -logFile -batchmode -buildTarget Linux64 -buildLinux64Player $FULL_BINARY_PATH -quit
print("SoccerAcademy project built for Linux!")

# Hyperparameters
Overwrite the `TrainingConfig.yaml` file to set hyperparameters for training

In [ ]:
%%writefile TrainingConfigPPO.yaml

# Behaviour Name
SoccerAcademy:

    # Common Trainer Configurations
    trainer: ppo
    summary_freq: 10000
    batch_size: 5120
    buffer_size: 512000
    hidden_units: 512
    learning_rate: 0.0003
    learning_rate_schedule: linear
    max_steps: 10000
    normalize: false
    num_layers: 3
    time_horizon: 1024

    # PPO-specific hyperparameters
    beta: 0.005
    epsilon: 0.2
    lambd: 0.95
    num_epoch: 3

    # Reward Signals
    reward_signals:
        extrinsic:
            strength: 1.0
            gamma: 0.99

    # Memory-enhanced Agents
    use_recurrent: false
    memory_size: 128
    sequence_length: 128

In [ ]:
%%writefile TrainingCurriculaA.yaml

# Behaviour Name
SoccerAcademy:

  # Curricula A
  measure: progress
  min_lesson_length: 100
  signal_smoothing: true
  thresholds: [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  parameters:
    opponent_speed: [0.0, 0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
    opponent_exist: [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

# Run Experiments using SoccerAcademy project
Train the model using ``mlagents-learn`` by groups of 200,000 training steps. Results are uploaded to Google Drive for visualization using TensorBoard

In [ ]:
!xvfb-run --auto-servernum mlagents-learn -h

In [ ]:
!xvfb-run --auto-servernum mlagents-learn $TRAINING_CONFIG_FILE --env=$FULL_BINARY_PATH --run-id=$EXPERIMENTS_NAME --num-envs $PARALLEL_EXECUTIONS --curriculum $CURRICULA_PATH --no-graphics --force

# Google Drive
Mount Google Drive as a virtual drive in Linux to upload experiment results

In [ ]:
# Mount google drive using oficial library
drive.mount('/content/google-drive')

In [ ]:
# Creates Google Drive folder
RESULTS_FOLDER = "/content/google-drive/My Drive/SoccerAcademy/{}".format(EXPERIMENTS_NAME)
!mkdir -p '$RESULTS_FOLDER'

# Copy summaries and models folders
!cp -r -v "/content/summaries" '$RESULTS_FOLDER'
!cp -r -v "/content/models" '$RESULTS_FOLDER'

print("Results uploaded to Google Drive!")